In [20]:
from SPARQLWrapper import SPARQLWrapper, RDFXML, JSON
import requests
import time

# Basic configuration
endpoint = "https://wb.pcp-on-web.de/store-oker/research/"
#query_path = "https://raw.githubusercontent.com/pcp-on-web/data-import/main/Import-Catalogus-Professorum-Lipsiensium/"


# This function counts the genrated tripples of the research ontolgy (data:)
def count_triples(endpoint,graph) :
    #count number of exiting triples
    sparql = SPARQLWrapper(endpoint+'sparql')
    sparql.setQuery('SELECT (COUNT(*) as ?Triples) FROM '+graph+' WHERE { ?s ?p ?o}')
    sparql.setReturnFormat(JSON)
    json_results = sparql.query().convert()
    return json_results['results']['bindings'][0]['Triples']['value']
    

# This function executes a sparql construct query and imports the data to the research ontolgy (data:)
def query_sparql(query, endpoint) :
    start_time = time.time()
    
    #perform construct Query
    sparql = SPARQLWrapper(endpoint+'sparql')
  
    #query += " LIMIT 10"
    sparql.setQuery(query)
    sparql.setReturnFormat(RDFXML)
    sparql.setOnlyConneg(True)
    resultsets = sparql.query().convert()
    result = resultsets.serialize(format='ttl')
    
    # Add PREFIXs to data to insert
    data=""
    data_graph="leer"
    text_lines = query.split("\n")
    info = text_lines[1]+"\n"+text_lines[2]
    for line in text_lines:
        if line.find("PREFIX ")==0:
           data += line + "\n"
           p = 0
           p = line.find("PREFIX data:") 
           if p == 0 :
                data_graph = line[line.find("<"):]
            
    # Add SPARQL INSERT to data to insert
    data += "INSERT DATA {\nGRAPH data:\n{\n"
    text_lines = result.split("\n")
    for line in text_lines:
        if line.find("@prefix ")==-1:
           data += line + "\n"
    # Close
    data += "}}"
   
    start_count = count_triples(endpoint,data_graph)
    
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(data)
    sparql.method = 'POST'
    sparql.query()

    end_count = count_triples(endpoint,data_graph)

    output =  "\n"+info+"\n"+start_count + "-" + end_count + "\n--- " + str(time.time() - start_time) + " seconds ---"
    return output


In [21]:
data = """
# Basic Import of Lecturers including relevant attributes and pictures
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cpm: <http://catalogus-professorum.org/cpm/2/>
PREFIX cpl: <http://catalogus-professorum.org/lipsiensium/>



CONSTRUCT   {

?lecturer rdf:type pcp:Lecturer.
?lecturer pcp:derivedFrom ?professor.

?lecturer pcp:gnd ?gnd.
?lecturer rdfs:label ?label .
?lecturer pcp:note ?note .
?lecturer pcp:furtherInformation ?furtherInformation .
?lecturer pcp:additionToSurname ?additionToSurname .
?lecturer pcp:alternativeWritingOfSurname ?alternativeWritingOfSurname .
?lecturer pcp:forename ?forename .
?lecturer pcp:surname ?surname .
?lecturer rdfs:seeAlso ?webLink .
?lecturer rdfs:seeAlso ?seeAlso .
?lecturer rdfs:seeAlso ?lectureLink .

#  ?lecturer pcp:idPerson ?idPerson .
#  ?lecturer pcp:birthname ?birthname .
#  ?lecturer pcp:corresponds ?corresponds .
#  ?lecturer pcp:hasEnrollment ?hasEnrollment .
#  ?lecturer pcp:hasOrigin ?hasOrigin .
#  ?lecturer pcp:isAttributedTo ?isAttributedTo .
#  ?lecturer pcp:latinForename ?latinForename .
#  ?lecturer pcp:latinSurname ?latinSurname .


?pcp_uri_picture pcp:derivedFrom ?picture .
?pcp_uri_picture rdf:type pcp:Picture.
?lecturer pcp:relatedDocument ?uri_pcp_picture.
?uri_pcp_picture pcp:pictureURL ?picture_url.
?uri_pcp_picture pcp:pictureArchive ?picture_archive.
?uri_pcp_picture pcp:pictureKey ?picture_key.


}
WHERE {
  
  GRAPH cpl: {  

      # Import all Professors relevant for Early Modern History
      ?professor a cpm:Professor.
      ?professor cpm:hasPeriod ?prof_birth.
      ?prof_birth rdf:type cpm:Birth.
      ?prof_birth cpm:date ?prof_birth_date .
      BIND (xsd:integer(substr(str(?prof_birth_date), 1, 4)) as ?prof_birth_year)       
      FILTER (!BOUND(?prof_birth_year) || ((?prof_birth_year>1450) && (?prof_birth_year<1800)))
      BIND (IRI(REPLACE(str(?professor),'http://catalogus-professorum.org/',str(data:))) AS ?lecturer). 

      #retreive basic properties from professor
      
      OPTIONAL {
        ?professor rdfs:label ?label_lang.
        BIND(str(?label_lang) as ?label) # Remove language tags from label
      }
      OPTIONAL {?professor cpm:note ?note.}
      OPTIONAL {?professor cpm:furtherInformation ?furtherInformation.}
      OPTIONAL {?professor cpm:webLink ?webLink.}
      OPTIONAL {?professor rdfs:seeAlso ?seeAlso.}
      OPTIONAL {?professor cpm:forename ?forename.}
      OPTIONAL {?professor cpm:surname ?surname.}
      OPTIONAL {?professor cpm:references ?references.}
      OPTIONAL {?professor cpm:literature ?literature.}
      OPTIONAL {?professor cpm:alternativeWritingOfSurname ?alternativeWritingOfSurname.}

      #Link to lecture repository / Vorlesungsverzeichnis
      OPTIONAL {?professor cpm:lectureLink ?lectureLink.}

      #Not used basic properties
      #OPTIONAL {?professor cpm:denomination ?denomination.}
      #OPTIONAL {?professor cpm:fullAcademicalTitle ?fullAcademicalTitle.}
      #OPTIONAL {?professor cpm:firstName ?firstName.}
      #OPTIONAL {?professor cpm:additionToSurname ?additionToSurname.}
      #OPTIONAL {?professor cpm:isTutor ?isTutor.}
      #OPTIONAL {?professor cpm:profession ?profession.}
      #OPTIONAL {?professor cpm:published ?published.}

      #retrieve gnd and convert to IRI
      OPTIONAL {
        ?professor cpm:gnd ?gnd_source.
        BIND(
          IF ( 
              #1
              contains(?gnd_source,"http"),
              #2
              IRI(REPLACE(str(?gnd_source),'http://','https://')),
              #3
                #3.1
                IF ( contains(?gnd_source," "),
                #3.2
                
                    #3.2.1
                    IF (contains(?gnd_source,"(DNB)"),
                    #3.2.2
                    IRI(CONCAT("https://d-nb.info/gnd/",REPLACE(?gnd_source," \\\\(DNB\\\\)",""))),
                    #3.2.3
                    ""),
                #3.3
                IRI(CONCAT("https://d-nb.info/gnd/",?gnd_source)))
            ) as ?gnd ).
        FILTER isIRI(?gnd).
        }

    OPTIONAL {?professor cpm:hasPicture ?picture.
        BIND (IRI(REPLACE(str(?picture),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_uri_picture). 

        OPTIONAL {?picture cpm:picture ?picture_url . }
        OPTIONAL {?picture cpm:pictureArchive ?picture_archive . }
        OPTIONAL {?picture cpm:pictureKey ?picture_key .}
    }
  }
}
"""
print(query_sparql(data,endpoint))


# Basic Import of Lecturers including relevant attributes and pictures
#-----
0-3986
--- 2.9427943229675293 seconds ---


In [22]:
data = """
# Stage of Life: Birth and Death of Lecturers
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cpm: <http://catalogus-professorum.org/cpm/2/>
PREFIX cpl: <http://catalogus-professorum.org/lipsiensium/>

CONSTRUCT   {

?pcp_period rdf:type ?pcp_period_type.
?pcp_period pcp:derivedFrom ?period.
?lecturer pcp:hasStageOfLife ?pcp_period.

?pcp_period pcp:date ?date.
?pcp_period rdfs:label ?period_label.
?pcp_period pcp:stageOfLifeLocation ?pcp_place.
?pcp_place rdf:type pcp:Location.
?pcp_place rdfs:label ?place_label.
?pcp_place pcp:derivedFrom ?place.


}
WHERE {
  
  GRAPH cpl: {  

      # Import all Professors relevant for Early Modern History
      ?professor a cpm:Professor.
      ?professor cpm:hasPeriod ?prof_birth.
      ?prof_birth rdf:type cpm:Birth.
      ?prof_birth cpm:date ?prof_birth_date .
      BIND (xsd:integer(substr(str(?prof_birth_date), 1, 4)) as ?prof_birth_year)       
      FILTER (!BOUND(?prof_birth_year) || ((?prof_birth_year>1450) && (?prof_birth_year<1800)))
      BIND (IRI(REPLACE(str(?professor),'http://catalogus-professorum.org/',str(data:))) AS ?lecturer). 
    
    
    
    ?professor cpm:hasPeriod ?period.
    ?period rdf:type ?period_type.
    FILTER(contains(str(?period_type),"Birth")||contains(str(?period_type),"Death"))
    BIND (IRI(REPLACE(str(?period),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_period).
    BIND (IRI(REPLACE(str(?period_type),str(cpm:),str(pcp:))) AS ?pcp_period_type).
    
    OPTIONAL {?period rdfs:label ?period_label.}
    OPTIONAL {?period cpm:date ?date.}
    OPTIONAL {?period cpm:periodPlace ?place.
        ?place rdfs:label ?place_label.
        BIND (IRI(REPLACE(str(?place),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_place). 
        }

    }
}
"""
print(query_sparql(data,endpoint))


# Stage of Life: Birth and Death of Lecturers
#-----
3986-7001
--- 1.3264555931091309 seconds ---


In [23]:
data = """
# Stage of Life: Ämter inkl. akademische Ämter und deren Orte
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cpm: <http://catalogus-professorum.org/cpm/2/>
PREFIX cpl: <http://catalogus-professorum.org/lipsiensium/>

CONSTRUCT   {

# Lecturer

?pcp_period rdf:type ?pcp_period_type.
?pcp_period pcp:derivedFrom ?period.
?lecturer pcp:hasStageOfLife ?pcp_period.

# Stage of Life dates
?pcp_period pcp:date ?date.
?pcp_period pcp:from ?from.
?pcp_period pcp:to ?to.
?pcp_period rdfs:label ?period_label.

# Stage of Life location
?pcp_period pcp:stageOfLifeLocation ?pcp_place.
?pcp_place rdf:type pcp:Location.
?pcp_place rdfs:label ?place_label.
?pcp_place pcp:derivedFrom ?place.

# Stage of Life Body
?pcp_period ?pcp_hasBody ?pcp_body.
?pcp_body rdf:type ?pcp_isBody.
?pcp_body pcp:derivedFrom ?body.



}
WHERE {
  
  GRAPH cpl: {  


      # Professors relevant for Early Modern History
      
      ?professor a cpm:Professor.
      ?professor cpm:hasPeriod ?prof_birth.
      ?prof_birth rdf:type cpm:Birth.
      ?prof_birth cpm:date ?prof_birth_date .
      BIND (xsd:integer(substr(str(?prof_birth_date), 1, 4)) as ?prof_birth_year)       
      FILTER (!BOUND(?prof_birth_year) || ((?prof_birth_year>1450) && (?prof_birth_year<1800)))
      BIND (IRI(REPLACE(str(?professor),'http://catalogus-professorum.org/',str(data:))) AS ?lecturer). 
    

      # Karriereschritte Ämter

      ?professor cpm:hasPeriod ?period.
      ?period rdf:type cpm:Office.
      BIND (IRI(REPLACE(str(?period),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_period).
    
      OPTIONAL {
        ?period rdfs:label ?period_label.
        BIND(
            IF( contains(?period_label,"Dekan")||
                contains(?period_label,"dekan")||
            contains(?period_label,"Rektor")||
            contains(?period_label,"rektor")||
            contains(?period_label,"Institut")||
            contains(?period_label,"institut")||
            contains(?period_label,"Rat")||
            contains(?period_label,"rat")||
            contains(?period_label,"Kommission")||
            contains(?period_label,"kommission")||
            contains(?period_label,"Senat")||
            contains(?period_label,"senat")||
            contains(?period_label,"Leiter")||
            contains(?period_label,"leiter"), pcp:AcademicOffice, pcp:Office)
            AS ?pcp_period_type).  
        
      }
        
      OPTIONAL {?period cpm:date ?date.}
      OPTIONAL {?period cpm:from ?from.}
      OPTIONAL {?period cpm:to ?to.}

      OPTIONAL {
      
        # Körperschaft eines Karriereschritts i.d.R. Fakultät
        
        ?period cpm:periodBody ?body.
        ?body rdf:type cpm:Faculty .
        BIND (pcp:Faculty AS ?pcp_isBody). 
        BIND (pcp:hasFaculty AS ?pcp_hasBody). 
        
        
         BIND (IRI(REPLACE(str(?body),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_body). 
        
        # Ort, wenn dieser existiert
        
        OPTIONAL {
            ?body cpm:bodyPlace ?place.
            ?place rdfs:label ?place_label.
            BIND (IRI(REPLACE(str(?place),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_place). 
        }

        # Ort, wenn dieser Karriereschritt an eine ander Körperschaft gebunden ist i.d.R. Universität

        OPTIONAL {
            ?body cpm:isPartOf ?university.
            ?university rdf:type cpm:Academy .
            ?university cpm:bodyPlace ?place.
            ?place rdfs:label ?place_label.
            BIND (IRI(REPLACE(str(?place),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_place). 
        }

      }

      OPTIONAL {
      
        # Körperschaft eines Karriereschritts i.d.R. Fakultät
        
        ?period cpm:periodBody ?body.
        ?body rdf:type cpm:Academy .
        BIND (pcp:University AS ?pcp_isBody). 
        BIND (pcp:hasUniversity AS ?pcp_hasBody). 
        
        
         BIND (IRI(REPLACE(str(?body),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_body). 
        
        # Ort, wenn dieser existiert
        
        OPTIONAL {
            ?body cpm:bodyPlace ?place.
            ?place rdfs:label ?place_label.
            BIND (IRI(REPLACE(str(?place),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_place). 
        }


      }



    }
}
"""
print(query_sparql(data,endpoint))


# Stage of Life: Ämter inkl. akademische Ämter und deren Orte
#-----
7001-13383
--- 3.5033512115478516 seconds ---


In [24]:
data = """
# Stage of Life: Karriereschritte inkl. Lehrstühle und deren Orte
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cpm: <http://catalogus-professorum.org/cpm/2/>
PREFIX cpl: <http://catalogus-professorum.org/lipsiensium/>

CONSTRUCT   {

# Lecturer
?pcp_period rdf:type ?pcp_period_type.
?pcp_period pcp:derivedFrom ?period.
?lecturer pcp:hasStageOfLife ?pcp_period.

# Stage of Life dates
?pcp_period pcp:date ?date.
?pcp_period pcp:from ?from.
?pcp_period pcp:to ?to.
?pcp_period rdfs:label ?period_label.

# Stage of Life location
?pcp_period pcp:stageOfLifeLocation ?pcp_place.
?pcp_place rdf:type pcp:Location.
?pcp_place rdfs:label ?place_label.
?pcp_place pcp:derivedFrom ?place.

# Stage of Life Body
?pcp_period ?pcp_hasBody ?pcp_body.
?pcp_body rdf:type ?pcp_isBody.
?pcp_body pcp:derivedFrom ?body.

}
WHERE {
  
  GRAPH cpl: {  
  
      # Relevante Professoren für die Frühe Neuzeit 
      
      ?professor a cpm:Professor.
      ?professor cpm:hasPeriod ?prof_birth.
      ?prof_birth rdf:type cpm:Birth.
      ?prof_birth cpm:date ?prof_birth_date .
      BIND (xsd:integer(substr(str(?prof_birth_date), 1, 4)) as ?prof_birth_year)       
      FILTER (!BOUND(?prof_birth_year) || ((?prof_birth_year>1450) && (?prof_birth_year<1800)))
      BIND (IRI(REPLACE(str(?professor),'http://catalogus-professorum.org/',str(data:))) AS ?lecturer). 
    

      # Karriereschritte 

      ?professor cpm:hasPeriod ?period.
      ?period rdf:type cpm:Career.
      BIND (IRI(REPLACE(str(?period),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_period).
      OPTIONAL {?period rdfs:label ?period_label.
        BIND(
              IF( contains(?period_label," oProf")||
                  contains(?period_label,"Professor mit Lehrstuhl") ||
                  contains(?period_label," oHonProf") ||
                  contains(?period_label," plaoProf") ||
                  contains(?period_label," o. Prof") ||
                  contains(?period_label," ordentliche Professorin") ||
                  contains(?period_label," ordentlicher Professor"), pcp:ChairProfessorship,
              IF( contains(?period_label," aoProf") ||
                  contains(?period_label," außerordentliche") ||
                  contains(?period_label,"Vertretungsprof") ||
                  contains(?period_label,"Gastprof") ||
                  contains(?period_label,"Gast-Prof") ||
                  contains(?period_label,"visting professor") ||
                  contains(?period_label,"HonProf") ||
                  contains(?period_label,"Honorar") ||
                  contains(?period_label," apl. Prof") ||
                  contains(?period_label," aplProf") ||
                  contains(?period_label," nplao Prof") || 
                  contains(?period_label," nplaoProf") || 
                  contains(?period_label,"Professor mit vollem Lehrauftrag") ||
                  contains(?period_label," ao. Prof") , pcp:AssociateProfessor,
              IF( contains(?period_label,"PD") ||
                  contains(?period_label,"Privat") ||
                  contains(?period_label,"privat") , pcp:PrivateLecturer,
              IF( contains(?period_label,"Lecturer") ||
                  contains(?period_label,"Lehrbeauftragt") ||
                  contains(?period_label,"dozent") ||
                  contains(?period_label,"Lehrauftrag") ||
                  contains(?period_label,"Dozent") , pcp:OfficeLecturer,
              IF( contains(?period_label,"Prof"), pcp:ChairProfessorship,
                      pcp:AcademicOffice)))))  AS ?pcp_period_type).  

      }

      OPTIONAL {?period cpm:date ?date.}
      OPTIONAL {?period cpm:from ?from.}
      OPTIONAL {?period cpm:to ?to.}

      OPTIONAL {
      
        # Körperschaft eines Karriereschritts hier Fakultät
        
        ?period cpm:periodBody ?body.
        ?body rdf:type cpm:Faculty .
        BIND (pcp:Faculty AS ?pcp_isBody). 
        BIND (pcp:hasFaculty AS ?pcp_hasBody). 

        
         BIND (IRI(REPLACE(str(?body),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_body). 
        
        # Ort, wenn dieser existiert
        
        OPTIONAL {
            ?body cpm:bodyPlace ?place.
            ?place rdfs:label ?place_label.
            BIND (IRI(REPLACE(str(?place),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_place). 
        }

        # Ort, wenn dieser Karriereschritt an eine ander Körperschaft gebunden ist i.d.R. Universität

        OPTIONAL {
            ?body cpm:isPartOf ?university.
            ?university rdf:type cpm:Academy .
            ?university cpm:bodyPlace ?place.
            ?place rdfs:label ?place_label.
            BIND (IRI(REPLACE(str(?place),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_place). 
        }

      }


      OPTIONAL {
      
        # Körperschaft eines Karriereschritts i.d.R. Fakultät
        
        ?period cpm:periodBody ?body.
        ?body rdf:type cpm:Academy .
        BIND (pcp:University AS ?pcp_isBody). 
        BIND (pcp:hasUniversity AS ?pcp_hasBody). 
        
        
         BIND (IRI(REPLACE(str(?body),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_body). 
        
        # Ort, wenn dieser existiert
        
        OPTIONAL {
            ?body cpm:bodyPlace ?place.
            ?place rdfs:label ?place_label.
            BIND (IRI(REPLACE(str(?place),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_place). 
        }

      }



    }
}
"""
print(query_sparql(data,endpoint))


# Stage of Life: Karriereschritte inkl. Lehrstühle und deren Orte
#-----
13383-19186
--- 3.0589981079101562 seconds ---


In [25]:
data = """
# Vervollständigung Stage of Life (ist ggf. doppelt zum vorherigen Import)
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cpm: <http://catalogus-professorum.org/cpm/2/>
PREFIX cpl: <http://catalogus-professorum.org/lipsiensium/>

CONSTRUCT   {

# Stage of Life dates
?pcp_period pcp:date ?date.
?pcp_period pcp:from ?from.
?pcp_period pcp:to ?to.
?pcp_period rdfs:label ?period_label.

}
WHERE {

  GRAPH data: {
    ?pcp_period rdf:type ?stage_type.
    FILTER contains(str(?pcp_period),'lipsiensium')
    FILTER ( ?stage_type=pcp:Office ||
        ?stage_type=pcp:AcademicOffice  ||
        ?stage_type=pcp:ChairProfessorship  ||
        ?stage_type=pcp:AssociateProfessor  ||
        ?stage_type=pcp:OfficeLecturer  ||
        ?stage_type=pcp:PrivateLecturer  
    )
    ?pcp_period pcp:derivedFrom ?period.

    GRAPH cpl: {  
        OPTIONAL {?period rdfs:label ?period_label.}
        OPTIONAL {?period cpm:date ?date.}
        OPTIONAL {?period cpm:from ?from.}
        OPTIONAL {?period cpm:to ?to.}
    }
  }
}
"""
print(query_sparql(data,endpoint))



# Vervollständigung Stage of Life
#-----
19186-19186
--- 2.6677091121673584 seconds ---


In [26]:
data = """
# Vervollständigung Fakultäten
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cpm: <http://catalogus-professorum.org/cpm/2/>
PREFIX cpl: <http://catalogus-professorum.org/lipsiensium/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

CONSTRUCT   {

# Fakultät
?pcp_faculty rdfs:label ?faculty_label.
?pcp_faculty owl:sameAs ?faculty_sameas.
?pcp_faculty rdfs:seeAlso ?faculty_link.
?pcp_faculty pcp:isPartOfUniversity ?pcp_university.


}
WHERE {
  GRAPH data: {
    ?pcp_faculty rdf:type pcp:Faculty.
    FILTER contains(str(?pcp_faculty),'lipsiensium')
    ?pcp_faculty pcp:derivedFrom ?body
    GRAPH cpl: {  
        OPTIONAL {?body rdfs:label ?faculty_label.}
        OPTIONAL {?body owl:sameAs ?faculty_sameas.}
        OPTIONAL {?body cpm:weblink ?faculty_link.}
    }


  }
}
"""
print(query_sparql(data,endpoint))

        


# Vervollständigung Fakultäten
#-----
19186-19198
--- 0.72648024559021 seconds ---


In [27]:
data = """
# Vervollständigung Universitäten
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cpm: <http://catalogus-professorum.org/cpm/2/>
PREFIX cpl: <http://catalogus-professorum.org/lipsiensium/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

CONSTRUCT   {

# University
?pcp_university rdfs:label ?university_label.
?pcp_university owl:sameAs ?university_sameas.
?pcp_university rdfs:seeAlso ?university_link.

# Universitätsort

?pcp_university pcp:locationCorporation ?pcp_place.
?pcp_place rdf:type pcp:Location.
?pcp_place rdfs:label ?place_label.


}
WHERE {
  GRAPH data: {
    ?pcp_university rdf:type pcp:University.
    FILTER contains(str(?pcp_university),'lipsiensium')
    ?pcp_university pcp:derivedFrom ?body
    GRAPH cpl: {  
        OPTIONAL {?body rdfs:label ?university_label.}
        OPTIONAL {?body owl:sameAs ?university_sameas.}
        OPTIONAL {?body cpm:weblink ?university_link.}
        OPTIONAL {
            ?body cpm:bodyPlace ?place.
            ?place rdfs:label ?place_label.
            BIND (IRI(REPLACE(str(?place),'http://catalogus-professorum.org/',str(data:))) AS ?pcp_place).             
        }
    }
  }
}
"""
print(query_sparql(data,endpoint))

        


# Vervollständigung Universitäten
#-----
19198-19278
--- 1.40877366065979 seconds ---


In [9]:
#Erledigt:
#cpm:Birth: 1849
#cpm:Death: 1503
#cpm:Office: 1660
#cpm:Career: 5929

#Offen:
#cpm:Membership: 194
#cpm:Study: 1843
#cpm:PeriodOfLife: 27
#cpm:Family: 2863
#cpm:Graduation: 1365
#cpm:SocialRole: 3602
#cpm:Qualification: 3611